In [ ]:
def import_all():
    import rqdatac
    import pandas as pd
    import statsmodels.api as sm
    from linearmodels.panel import FamaMacBeth
    import csv
    import numpy as np
    rqdatac.init()
    start_date = pd.to_datetime('2018-01-01')
    end_date = pd.to_datetime('2020-12-31')
    instrument_type = 'Convertible'
    factor_name = '第二产业增加值占GDP比重(现价)'
    remaining_time_to_mature ='短期 (1-3年)'
    rolling_window = 90

import rqdatac
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import FamaMacBeth
import csv
import numpy as np


/opt/anaconda3/envs/rqsdk/lib/python3.9/site-packages/rqdatac/client.py:218: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)
/opt/anaconda3/envs/rqsdk/lib/python3.9/site-packages/rqdatac/client.py:257: UserWarning: Your account will be expired after  29 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


## Selecting Valied Bond

In [183]:
def get_bonds_poll(start_date:str,end_date:str,instrument_type:str) -> pd.DataFrame:
    '''
    最基本的筛选时间内所有股票基本信息的函数
    返回： 一个带有 order book id，delisted date等等的 data frame
     '''
    all_instruments = rqdatac.all_instruments(instrument_type)
    all_instruments['listed_date'] = pd.to_datetime(all_instruments['listed_date'], errors='coerce')
    all_instruments['de_listed_date'] = pd.to_datetime(all_instruments['de_listed_date'], errors='coerce')
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    mask = (all_instruments['listed_date'] <= end_date) & (all_instruments['de_listed_date'].isna() | (all_instruments['de_listed_date'] >= start_date))
    all_instruments = all_instruments.loc[mask]
    print("可转债数量:", len(all_instruments['order_book_id']))
    return all_instruments
# all_instruments = get_bonds_poll(start_date, end_date, instrument_type).sample(10) # 这里一定记得删除！！！
# print(all_instruments.info())


可转债数量: 449
<class 'pandas.core.frame.DataFrame'>
Index: 449 entries, 41 to 1098
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   de_listed_date  351 non-null    datetime64[ns]
 1   type            449 non-null    object        
 2   order_book_id   449 non-null    object        
 3   listed_date     449 non-null    datetime64[ns]
 4   exchange        449 non-null    object        
 5   status          449 non-null    object        
 6   symbol          449 non-null    object        
 7   round_lot       449 non-null    int64         
 8   market_tplus    449 non-null    int64         
 9   trading_code    449 non-null    object        
 10  trading_hours   449 non-null    object        
 11  stock_code      449 non-null    object        
 12  maturity_date   449 non-null    object        
dtypes: datetime64[ns](2), int64(2), object(9)
memory usage: 49.1+ KB
None


In [184]:
def get_return_rate(all_instruments, start_date, end_date):
    '''
    输入所有可用的 order book id
    起止时间
    
    输出：一个以order book id 和 date 为index，  return 30 ,daily return 为column的 data frame
    至此dependent已处理完毕
    '''
    all_prices = rqdatac.get_price(all_instruments['order_book_id'], start_date, end_date + pd.DateOffset(days=23)).reset_index() 
    # 扩展时间范围
    all_prices['date'] = pd.to_datetime(all_prices['date'])
    all_prices.set_index(['order_book_id', 'date'], inplace=True)

    all_prices['future_price'] = all_prices.groupby(level='order_book_id')['close'].shift(-22)
    all_prices['return_30'] = (all_prices['future_price'] - all_prices['close']) / all_prices['close']
    all_prices['daily_return'] = all_prices.groupby('order_book_id')['close'].pct_change()

    dependent = all_prices[['return_30','daily_return']].dropna()
    return dependent
    
# dependent = get_return_rate(all_instruments,start_date,end_date)
# print(dependent)


                          return_30  daily_return
order_book_id date                               
110030.XSHG   2018-01-03   0.025033      0.000569
              2018-01-04   0.029356      0.001328
              2018-01-05  -0.016737      0.035417
              2018-01-08  -0.024964      0.003841
              2018-01-09  -0.019051     -0.005284
...                             ...           ...
132022.XSHG   2020-12-16  -0.008317      0.006979
              2020-12-17  -0.005941      0.000000
              2020-12-18  -0.005446      0.000000
              2020-12-21  -0.009755      0.004851
              2020-12-22  -0.005760     -0.007784

[129384 rows x 2 columns]


In [185]:
def econ_get_factor_value_forward_fill(factor_name,start_date,end_date):# 这里什么日期当作要查看的日期
    '''这是用econ get factor 向前填充得到的因子值
    输入： factor name
    输出：
    index为发行日期‘date’的数据
    column是向前填充的标准化之后的因子值 注意：一定将因子值的名称改为‘value’

    '''
    print(f'testing: {factor_name}')
    factor_df = rqdatac.econ.get_factors(factor_name, start_date, end_date).reset_index()
    factor_df['date'] = factor_df['info_date']
    factor_df = factor_df[['date', 'value']]
    factor_df['value'] = (factor_df['value']-factor_df['value'].mean()) /factor_df['value'].std()
    factor_df.set_index('date', inplace=True)
    date_range = pd.date_range(start_date, end_date, freq='d')
    factor_df = factor_df.reindex(date_range, method='ffill').dropna()
    factor_df.index.names = ['date']
    return factor_df
# independent = econ_get_factor_value_forward_fill(factor_name,start_date,end_date)
# print(independent)


               value
date                
2018-02-28  0.648074
2018-03-01  0.648074
2018-03-02  0.648074
2018-03-03  0.648074
2018-03-04  0.648074
...              ...
2020-12-27 -1.151684
2020-12-28 -1.151684
2020-12-29 -1.151684
2020-12-30 -1.151684
2020-12-31 -1.151684

[1038 rows x 1 columns]


In [186]:
def prepare_time_regression_df (dependent, independent):
    '''
    将因子值与收益合并
    使得因子值在每个时间，每只股票上都有对齐
    
    输入：一个 order book id， date为index的大收益表
    一个 date 为index的收益表'''
    factor_df = pd.merge(
            dependent.reset_index(), 
            independent.reset_index(), # reset_index() 让 'date' 变回普通列以便合并
            on='date', 
            how='left'
        )
    factor_df = factor_df.dropna(subset=['value'])
    return factor_df

# time_regression_df = prepare_time_regression_df(dependent,independent)

       order_book_id       date  return_30  daily_return     value
35       110030.XSHG 2018-02-28   0.008097      0.001491  0.648074
36       110030.XSHG 2018-03-01   0.008192     -0.000279  0.648074
37       110030.XSHG 2018-03-02   0.010456     -0.002793  0.648074
38       110030.XSHG 2018-03-05   0.009308      0.002987  0.648074
39       110030.XSHG 2018-03-06   0.008707      0.004840  0.648074
...              ...        ...        ...           ...       ...
129379   132022.XSHG 2020-12-16  -0.008317      0.006979 -1.151684
129380   132022.XSHG 2020-12-17  -0.005941      0.000000 -1.151684
129381   132022.XSHG 2020-12-18  -0.005446      0.000000 -1.151684
129382   132022.XSHG 2020-12-21  -0.009755      0.004851 -1.151684
129383   132022.XSHG 2020-12-22  -0.005760     -0.007784 -1.151684

[127270 rows x 5 columns]


In [187]:
def roll_data_and_regress(time_regression_df,rolling_window):
        '''
        输入： column 有order book id 和date 和daily return 和value的大表
        创建滚动窗并在滚动窗内计算beta值
        输出 column 有 （order book id 和 date ）和每股每天因子值的data frame
        
        
        '''
        results_list = []
        window = rolling_window
        for cb, group in time_regression_df.groupby('order_book_id'):
            if len(group) < window:
                # print(f"Skipping {cb}: Not enough data for a full window ({len(group)})")
                continue

            # Inner loop calculates the rolling beta for the current stock
            for i in range(window, len(group) + 1):
                # Get the current window of data
                df_window = group.iloc[i-window:i]

                if df_window.isnull().values.any():
                    continue
                    
                # Perform the OLS regression
                y = df_window['daily_return']
                X = sm.add_constant(df_window['value'])
                model_fit = sm.OLS(y, X).fit()
                beta_value = model_fit.params.get('value')
                
                # 2. Get the end date of the current window
                # The date is the first level of the group's index
                end_date = df_window['date'].iloc[-1]
                
                # 3. Append a dictionary with ALL info to the list
                results_list.append({
                    'order_book_id': cb,
                    'date': end_date,
                    'beta': beta_value  # Use the factor_name as the column title
                })

        # 4. After all loops are done, create the DataFrame from the list of dictionaries
        rolling_betas = pd.DataFrame(results_list)
        return rolling_betas
# rolling_betas = roll_data_and_regress(time_regression_df, rolling_window)
# print(rolling_betas.info())

Skipping 110039.XSHG: Not enough data for a full window (70)
Skipping 110072.XSHG: Not enough data for a full window (64)
Skipping 110073.XSHG: Not enough data for a full window (82)
Skipping 110074.XSHG: Not enough data for a full window (60)
Skipping 110075.XSHG: Not enough data for a full window (35)
Skipping 110076.XSHG: Not enough data for a full window (19)
Skipping 110077.XSHG: Not enough data for a full window (3)
Skipping 113037.XSHG: Not enough data for a full window (85)
Skipping 113038.XSHG: Not enough data for a full window (69)
Skipping 113039.XSHG: Not enough data for a full window (62)
Skipping 113040.XSHG: Not enough data for a full window (26)
Skipping 113041.XSHG: Not enough data for a full window (17)
Skipping 113594.XSHG: Not enough data for a full window (76)
Skipping 113595.XSHG: Not enough data for a full window (84)
Skipping 113596.XSHG: Not enough data for a full window (82)
Skipping 113597.XSHG: Not enough data for a full window (81)
Skipping 113598.XSHG: Not

In [188]:
def add_maturity_group_tag(rolling_betas,all_instruments):
    '''输入一个正常的rolling beta的beta 双index 列表，和all——instrument 提供delisted date
        返回一个添加了‘maturity——group——tag的’ rolling beta
    '''

    maturity_lists = all_instruments[['order_book_id','de_listed_date','maturity_date']].copy()
    rolling_betas = pd.merge(rolling_betas,maturity_lists,on = 'order_book_id',how = 'left')
    rolling_betas['effective_end_date'] = rolling_betas['de_listed_date'].combine_first(rolling_betas['maturity_date'])
    rolling_betas['remaining_maturity'] = (rolling_betas['effective_end_date'] - rolling_betas['date']).dt.days / 365.25


    bins = [0, 3, 5, 100]  # 100可以看作是一个足够大的数，代表无穷远
    labels = ['短期 (1-3年)', '中期 (3-5年)', '长期 (5+年)']
    rolling_betas['maturity_group_tag'] = pd.cut(rolling_betas['remaining_maturity'], bins=bins, labels=labels, right=False)

    return rolling_betas[['date','order_book_id','beta','maturity_group_tag']]

# rolling_betas_with_timegroup_tag = add_maturity_group_tag(rolling_betas,all_instruments)
# print(rolling_betas_with_timegroup_tag.sample(50))


            date order_book_id      beta maturity_group_tag
76110 2020-03-03   128059.XSHE  0.005449          短期 (1-3年)
87369 2019-06-21   132011.XSHG  0.002640          中期 (3-5年)
87743 2018-07-18   132012.XSHG -0.000880          短期 (1-3年)
45583 2020-10-23   123025.XSHE  0.001367          中期 (3-5年)
1047  2018-12-25   110032.XSHG -0.001201          短期 (1-3年)
965   2018-08-23   110032.XSHG  0.001158          短期 (1-3年)
3219  2018-12-12   110041.XSHG  0.000035          短期 (1-3年)
19758 2018-11-12   113502.XSHG -0.000010          中期 (3-5年)
39231 2020-02-20   123007.XSHE  0.003520          短期 (1-3年)
58328 2020-10-22   128018.XSHE -0.000418          短期 (1-3年)
16546 2019-08-08   113019.XSHG  0.002423          短期 (1-3年)
37151 2018-10-31   123002.XSHE  0.000690           长期 (5+年)
85264 2019-01-03   132007.XSHG  0.000379          短期 (1-3年)
52319 2020-12-08   127008.XSHE  0.000100          短期 (1-3年)
36706 2020-08-26   120004.XSHE -0.000045          短期 (1-3年)
6928  2019-07-26   110050.XSHG -0.001087

In [189]:
remaining_time_to_mature ='短期 (1-3年)'
def select_wanted_rolling_data(rolling_betas_with_timegroup_tag,remaining_time_to_mature):
    '''选出目标债'''
    return rolling_betas_with_timegroup_tag.loc[rolling_betas_with_timegroup_tag['maturity_group_tag'] == remaining_time_to_mature]
# rolling_betas = select_wanted_rolling_data(rolling_betas_with_timegroup_tag,remaining_time_to_mature)
# print(rolling_betas)

            date order_book_id      beta maturity_group_tag
0     2018-07-10   110030.XSHG -0.000481          短期 (1-3年)
1     2018-07-11   110030.XSHG -0.000543          短期 (1-3年)
2     2018-07-12   110030.XSHG -0.000525          短期 (1-3年)
3     2018-07-13   110030.XSHG -0.000487          短期 (1-3年)
4     2018-07-16   110030.XSHG -0.000537          短期 (1-3年)
...          ...           ...       ...                ...
91302 2020-12-16   132022.XSHG  0.000245          短期 (1-3年)
91303 2020-12-17   132022.XSHG  0.000223          短期 (1-3年)
91304 2020-12-18   132022.XSHG  0.000224          短期 (1-3年)
91305 2020-12-21   132022.XSHG  0.000178          短期 (1-3年)
91306 2020-12-22   132022.XSHG  0.000205          短期 (1-3年)

[47357 rows x 4 columns]


In [190]:
def do_fama_macbeth(rolling_betas,dependent,factor_name,remaining_time_to_mature):
    exog = rolling_betas.set_index(['order_book_id','date'],inplace=False)
    dependent = dependent[['return_30']]

    exog = exog[['beta']]

    common_index = dependent.index.intersection(exog.index)

    dependent = dependent.loc[common_index]
    exog = sm.add_constant(exog.loc[common_index])
    try:
                model = FamaMacBeth(dependent, exog)
                results = model.fit()
                print(results)
                p_value = results.pvalues['beta']
                t_stat = results.tstats['beta']

                if p_value < 0.07:
                    data = f'factor_name:  {factor_name}    p_value: {p_value}  t_value:{t_stat}\n'

                    with open( f'Result_file_{remaining_time_to_mature}.csv','a') as file:
                        file.write(data)
                else:
                    data = f'factor_name:  {factor_name}    p_value: {p_value}  t_value:{t_stat}\n'

                    with open(f'Result_file_dropped_{remaining_time_to_mature}.csv','a') as file:
                        file.write(data)

                    
                    
    except ValueError as e:
        print("错误:", e)
        print("每个 order_book_id 的 exog 观测数:", exog.groupby(level='order_book_id').size())
# result = do_fama_macbeth(rolling_betas,dependent)


                             FamaMacBeth Estimation Summary                            
Dep. Variable:                     return_30   R-squared:                       -0.0124
Estimator:                       FamaMacBeth   R-squared (Between):             -0.0228
No. Observations:                      47357   R-squared (Within):              -0.0118
Date:                       Thu, Jul 17 2025   R-squared (Overall):             -0.0124
Time:                               17:18:25   Log-likelihood                 4.084e+04
Cov. Estimator:    Fama-MacBeth Standard Cov                                           
                                               F-statistic:                     -580.70
Entities:                                234   P-value                           1.0000
Avg Obs:                              202.38   Distribution:                 F(1,47355)
Min Obs:                              2.0000                                           
Max Obs:                        

In [ ]:
def main_test_for_econ_get_factors(start_date,end_date,instrument_type,factor_name,remaining_time_to_mature,rolling_window):
    try:
        
        all_instruments = get_bonds_poll(start_date, end_date, instrument_type)
        dependent = get_return_rate(all_instruments,start_date,end_date)
        independent = econ_get_factor_value_forward_fill(factor_name,start_date,end_date)
        time_regression_df = prepare_time_regression_df(dependent,independent)
        rolling_betas = roll_data_and_regress(time_regression_df, rolling_window)
        #cb only
        rolling_betas_with_timegroup_tag = add_maturity_group_tag(rolling_betas,all_instruments)
        rolling_betas = select_wanted_rolling_data(rolling_betas_with_timegroup_tag,remaining_time_to_mature)
        result = do_fama_macbeth(rolling_betas,dependent,factor_name,remaining_time_to_mature)
    except:
        data = f'factor_name:  {factor_name}    error\n'
        with open(f'Result_file_error_{remaining_time_to_mature}.csv','a') as file:
                        file.write(data)
'''start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2020-12-31')
instrument_type = 'Convertible'
factor_name = '第二产业增加值占GDP比重(现价)'
remaining_time_to_mature ='短期 (1-3年)'
rolling_window = 90'''
main_test_for_econ_get_factors(start_date,end_date,instrument_type,factor_name,remaining_time_to_mature,rolling_window)

可转债数量: 449
Skipping 110039.XSHG: Not enough data for a full window (70)
Skipping 110072.XSHG: Not enough data for a full window (64)
Skipping 110073.XSHG: Not enough data for a full window (82)
Skipping 110074.XSHG: Not enough data for a full window (60)
Skipping 110075.XSHG: Not enough data for a full window (35)
Skipping 110076.XSHG: Not enough data for a full window (19)
Skipping 110077.XSHG: Not enough data for a full window (3)
Skipping 113037.XSHG: Not enough data for a full window (85)
Skipping 113038.XSHG: Not enough data for a full window (69)
Skipping 113039.XSHG: Not enough data for a full window (62)
Skipping 113040.XSHG: Not enough data for a full window (26)
Skipping 113041.XSHG: Not enough data for a full window (17)
Skipping 113594.XSHG: Not enough data for a full window (76)
Skipping 113595.XSHG: Not enough data for a full window (84)
Skipping 113596.XSHG: Not enough data for a full window (82)
Skipping 113597.XSHG: Not enough data for a full window (81)
Skipping 11359

In [197]:
if __name__ == '==main==':
    import_all()
    all_instruments = get_bonds_poll(start_date, end_date, instrument_type)
    dependent = get_return_rate(all_instruments,start_date,end_date)
    independent = econ_get_factor_value_forward_fill(factor_name,start_date,end_date)
    time_regression_df = prepare_time_regression_df(dependent,independent)
    rolling_betas = roll_data_and_regress(time_regression_df, rolling_window)
    #cb only
    rolling_betas_with_timegroup_tag = add_maturity_group_tag(rolling_betas,all_instruments)
    rolling_betas = select_wanted_rolling_data(rolling_betas_with_timegroup_tag,remaining_time_to_mature)
    result = do_fama_macbeth(rolling_betas,dependent)